# Build a Supervisor Multi-Agent System for Financial Research and Data Analysis with LangGraph

In this project we will be building a Multi-Agent System following the Supervisor Architecture. The Supervisor Agent will supervised two sub-agents, 'Financial Researcher' and 'Coder'to help get useful financial data, analyze and visualize them using graphs.

![](https://i.imgur.com/VnGr1n3.png)


### Supervisor Multi-Agent System for Financial Research and Data Analysis

This project focuses on building a **Supervisor Multi-Agent System for Financial Research and Data Analysis**. The system uses a supervisor agent design, where a **Supervisor Agent** delegates tasks to specialized sub-agents to efficiently handle complex financial queries and data analysis tasks. The workflow is as follows:

1. **Supervisor Agent**:
   - Analyzes the user's query to determine the required actions.
   - Dynamically delegates tasks to one of the following sub-agents:
     - **Financial Researcher Agent**: Responsible for retrieving financial data.
     - **Coder & Visualization Agent**: Responsible for data processing and visualization.
   - Maintains control over the process by monitoring the state and progress of both sub-agents, ensuring the query is fully resolved.

2. **Financial Researcher Agent**:
   - Uses **financial and web search tools** to gather relevant data and insights based on the user query.
   - Returns the collected data to the Supervisor Agent for further processing.
   - Example: Retrieves financial metrics such as ROE (Return on Equity) for companies like NVIDIA, Apple, Intel, Microsoft, and Amazon.

3. **Coder & Visualization Agent**:
   - Takes the data provided by the Financial Researcher Agent or directly delegated by the Supervisor Agent.
   - Uses **programming tools**, in this case Python, to:
     - Process the data.
     - Generate visualizations like bar charts, line graphs, etc., to represent the results.
   - Sends the generated output back to the Supervisor Agent.

4. **Dynamic Task Coordination**:
   - The Supervisor Agent coordinates between the sub-agents iteratively, based on the task's progress and the current state.
   - Continuously evaluates whether additional data collection, computation, or visualization is needed.
   - Ensures seamless integration between data retrieval and analysis tasks.

5. **Final Response**:
   - Once all tasks are completed, the Supervisor Agent compiles the outputs from the sub-agents into a cohesive final response.
   - Example: Produces a bar chart displaying the ROE values for the selected companies, fulfilling the user's query.


## Install OpenAI, LangGraph and LangChain dependencies

In [0]:
!pip install langchain==0.3.14
!pip install langchain-openai==0.3.0
!pip install langchain-community==0.3.14
!pip install langgraph==0.2.64
!pip install langchain-experimental==0.3.4
!pip install yfinance==0.2.51

## Install OpenBB

In [0]:
!pip install openbb[all]

## Enter Open AI API Key

In [0]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

## Enter OpenBB Key

Get a free API key from [here](https://my.openbb.co/app/platform/pat)

In [0]:
from getpass import getpass

OPENBB_PAT = getpass('Enter OpenBB Personal Access Token (PAT): ')

## Enter Tavily Search API Key

Get a free API key from [here](https://tavily.com/#api)

In [0]:
TAVILY_API_KEY = getpass('Enter Tavily Search API Key: ')

## Setup Environment Variables

In [0]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

In [0]:
import os
os.environ['RUFF_CACHE_DIR'] = '/tmp/ruff_cache'

from openbb import obb

In [0]:
1

In [0]:
from openbb import obb
# takes 1 min to setup
# obb.account.login(pat=OPENBB_PAT)
obb.user.credentials.fmp_api_key = dbutils.secrets.get(scope="AgenticAI", key="fmp_api_key")
obb.user.credentials.polygon_api_key = dbutils.secrets.get(scope="AgenticAI", key="polygon_api_key")

## Create Financial Tools

**Financial Analysis Tools**:
   The system integrates multiple tools to get useful financial data and metrics:
   - **SEARCH_WEB**: Fetches general stock market information from the web.
   - **GET_STOCK_FUNDAMENTAL_INDICATOR_METRICS**: Provides insights into key financial metrics such as P/E ratio, ROE, etc.
   - **GET_STOCK_NEWS**: Extracts the latest news and updates related to stocks or markets.
   - **GET_GENERAL_MARKET_DATA**: Fetches data on overall market trends and performance.
   - **GET_STOCK_TICKER**: Validates and fetches stock ticker symbols based on user queries.
   - **GET_STOCK_PRICE_METRICS**: Retrieves price trends, performance, and metrics for specific stocks.

In [0]:
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_core.tools import tool
import json
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

tavily_search = TavilySearchAPIWrapper()

@tool
def search_web(query: str, num_results=10) -> list:
    """Search the web for a query. Userful for general information or general news"""
    results = tavily_search.raw_results(query=query,
                                        max_results=num_results,
                                        search_depth='advanced',
                                        include_answer=False,
                                        include_raw_content=True)
    return results

@tool
def get_stock_ticker_symbol(stock_name: str) -> str:
    """Get the symbol, name and CIK for any publicly traded company"""
    # Use OpenBB to search for stock ticker symbol and company details by name.
    # The provider "sec" fetches data from the U.S. Securities and Exchange Commission (SEC).
    res = obb.equity.search(stock_name, provider="sec")

    # Convert the result to a DataFrame and format it as markdown for readability.
    stock_ticker_details = res.to_df().to_markdown()

    # Prepare the output with the stock details.
    output = """Here are the details of the company and its stock ticker symbol:\n\n""" + stock_ticker_details
    return output

@tool
def get_stock_price_metrics(stock_ticker: str) -> str:
    """Get historical stock price data, stock price quote and price performance data
       like price changes for a specific stock ticker"""

    # Fetch the latest stock price quote using "cboe" provider.
    res = obb.equity.price.quote(stock_ticker, provider='cboe')
    price_quote = res.to_df().to_markdown()

    # Retrieve stock price performance metrics (e.g., percentage change) using "finviz" provider.
    res = obb.equity.price.performance(symbol=stock_ticker, provider='finviz')
    price_performance = res.to_df().to_markdown()

    # Fetch historical price data for the past year using "yfinance" provider.
    end_date = datetime.now()
    start_date = (end_date - timedelta(days=365)).strftime("%Y-%m-%d")
    res = obb.equity.price.historical(symbol=stock_ticker, start_date=start_date,
                                      interval='1d', provider='yfinance')
    price_historical = res.to_df().to_markdown()

    # Combine the results into a formatted output.
    output = ("""Here are the stock price metrics and data for the stock ticker symbol """ + stock_ticker + """: \n\n""" +
              "Price Quote Metrics:\n\n" + price_quote +
              "\n\nPrice Performance Metrics:\n\n" + price_performance +
              "\n\nPrice Historical Data:\n\n" + price_historical)
    return output

@tool
def get_stock_fundamental_indicator_metrics(stock_ticker: str) -> str:
    """Get fundamental indicator metrics for a specific stock ticker"""

    # Retrieve fundamental financial ratios (e.g., P/E ratio, ROE) using "fmp" provider.
    res = obb.equity.fundamental.ratios(symbol=stock_ticker, period='annual',
                                        limit=10, provider='fmp')
    fundamental_ratios = res.to_df().to_markdown()

    # Fetch additional fundamental metrics (e.g., EBITDA, revenue growth) using "yfinance" provider.
    res = obb.equity.fundamental.metrics(symbol=stock_ticker, period='annual',
                                        limit=10, provider='yfinance')
    fundamental_metrics = res.to_df().to_markdown()

    # Combine fundamental ratios and metrics into a single output.
    output = ("""Here are the fundamental indicator metrics and data for the stock ticker symbol """ + stock_ticker + """: \n\n""" +
              "Fundamental Ratios:\n\n" + fundamental_ratios +
              "\n\nFundamental Metrics:\n\n" + fundamental_metrics)
    return output

@tool
def get_stock_news(stock_ticker: str) -> str:
    """Get news article headlines for a specific stock ticker"""

    # Define the date range to fetch news (last 45 days).
    end_date = datetime.now()
    start_date = (end_date - timedelta(days=45)).strftime("%Y-%m-%d")

    # Retrieve news headlines for the stock using "tmx" provider.
    res = obb.news.company(symbol=stock_ticker, start_date=start_date, provider='tmx', limit=50)
    news = res.to_df()

    # Extract relevant columns (symbols and titles) and format as markdown.
    news = news[['symbols', 'title']].to_markdown()

    # Prepare the output with the news headlines.
    output = ("""Here are the recent news headlines for the stock ticker symbol """ + stock_ticker + """: \n\n""" + news)
    return output

@tool
def get_general_market_data() -> str:
    """Get general data and indicators for the whole stock market including,
       most actively traded stocks based on volume, top price gainers and top price losers.
       Useful when you want an overview of the market and what stocks to look at."""

    # Retrieve the most actively traded stocks using "yfinance" provider.
    res = obb.equity.discovery.active(sort='desc', provider='yfinance', limit=15)
    most_active_stocks = res.to_df().to_markdown()

    # Fetch the top price gainers using "yfinance" provider.
    res = obb.equity.discovery.gainers(sort='desc', provider='yfinance', limit=15)
    price_gainers = res.to_df().to_markdown()

    # Retrieve the top price losers using "yfinance" provider.
    res = obb.equity.discovery.losers(sort='desc', provider='yfinance', limit=15)
    price_losers = res.to_df().to_markdown()

    # Combine the market data into a single formatted output.
    output = ("""Here's some detailed information of the stock market which includes most actively traded stocks, gainers and losers:\n\n""" +
              "Most actively traded stocks:\n\n" + most_active_stocks +
              "\n\nTop price gainers:\n\n" + price_gainers +
              "\n\nTop price losers:\n\n" + price_losers)
    return output


## Create Coding Tools

Create a tool to run python code

In [0]:
from langchain_experimental.utilities import PythonREPL
from langchain_core.tools import tool
from typing import Annotated

repl = PythonREPL()

@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute code, generate charts."],
):
    """Use this to execute python code and do math. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n```\nCODE OUTPUT:\n {result}"
    return result_str

### Create Agent Supervisor

It will use LLM with structured output routing to choose the next worker node (sub-agent) OR finish processing.

### Explanation: How the Supervisor Agent Works

The notebook defines a **Supervisor Agent** that manages a workflow between two specialized sub-agents: a "researcher" (for financial data gathering) and a "coder" (for code execution and visualization).

#### How the Supervisor Agent is Built and Used

- **LLM Integration**: The agent uses OpenAI's GPT-4o model via `ChatOpenAI` to interpret user requests and decide which sub-agent should act next.
- **Structured Output**: The LLM is prompted with a system message (`SUPERVISOR_AGENT_PROMPT`) that describes the roles of the "researcher" and "coder". It is instructed to analyze the conversation and select the next worker or finish the process.
- **Router Schema**: The `Router` TypedDict defines the possible next steps: "researcher", "coder", or "FINISH".
- **Supervisor Node Function**: The `supervisor_node` function takes the current conversation state, appends the system prompt, and invokes the LLM with structured output. Based on the LLM's response, it routes the workflow to the appropriate sub-agent or ends the process.
- **State Management**: The agent maintains a list of messages and the current "next" worker in the state, ensuring context is preserved across steps.

#### How the Agent Works

1. **Receives User Request**: The user provides a request (e.g., "Get the stock price details of Nvidia and Intel and display it as a line chart").
2. **Supervisor Decides**: The supervisor agent analyzes the request and conversation history, then decides whether the "researcher" or "coder" should act next.
3. **Routes to Sub-Agent**: The chosen sub-agent performs its task (e.g., data gathering or code execution) and reports back.
4. **Iterative Process**: The supervisor continues to route between sub-agents as needed, based on their outputs, until the task is complete.
5. **Finish**: When the workflow is done, the supervisor returns "FINISH" to end the process.

This design enables dynamic, multi-step workflows where each sub-agent specializes in a part of the task, and the supervisor coordinates the overall process using LLM-driven reasoning and structured output routing.

### Explanation of the Supervisor Agent Code

This code defines the logic for a "Supervisor Agent" that manages a workflow between two sub-agents: a "researcher" and a "coder".

#### 1. Supervisor Prompt
- `SUPERVISOR_AGENT_PROMPT` is a system prompt that instructs the LLM (Large Language Model) to act as a supervisor.
- The supervisor's job is to decide, based on the conversation and user request, which worker ("researcher" or "coder") should act next.
- The supervisor is told to analyze results after each worker acts and to finish the process by responding with "FINISH" when appropriate.

#### 2. Router and State Schemas
- `Router` is a TypedDict that restricts the supervisor's output to one of three options: "researcher", "coder", or "FINISH".
- `State` is a TypedDict that keeps track of the conversation messages and the next worker to act.

#### 3. LLM Initialization
- `llm` is an instance of `ChatOpenAI` using the "gpt-4o" model with deterministic output (`temperature=0`).

#### 4. Supervisor Node Function
- `supervisor_node` is a function that:
  - Prepares the message history by adding the supervisor prompt to the conversation.
  - Calls the LLM with structured output, expecting a response that matches the `Router` schema.
  - Determines the next worker to act (`goto`). If the response is "FINISH", it sets `goto` to `END` (a special marker).
  - Returns a `Command` object that tells the workflow which worker to call next and updates the state accordingly.

This setup enables dynamic, LLM-driven routing between specialized agents in a multi-step workflow.

In [0]:
# from langchain_openai import ChatOpenAI

from typing import Literal, Annotated
from typing_extensions import TypedDict
from langgraph.graph import MessagesState, END
from langgraph.types import Command
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI

members = ["researcher", "coder"]

SUPERVISOR_AGENT_PROMPT = f"""You are a supervisor tasked with managing a conversation between the following workers:
                              {members}.

                              Given the following user request, respond with the worker to act next.
                              Each worker will perform a task and respond with their results and status.
                              Analyze the results carefully and decide which worker to call next accordingly.
                              Remember researcher agent can search for information and coder agent can code.
                              When finished, respond with FINISH."""


class Router(TypedDict):
    next: Literal["researcher", "coder", "FINISH"]

class State(TypedDict):
    messages: Annotated[list, add_messages]
    next: str

llm = ChatOpenAI(model="gpt-4o", temperature=0)

def supervisor_node(state: State) -> Command[Literal["researcher", "coder", "__end__"]]:
    messages = [{"role": "system", "content": SUPERVISOR_AGENT_PROMPT},] + state["messages"]
    response = llm.with_structured_output(Router).invoke(messages)
    goto = response["next"]
    if goto == "FINISH":
        goto = END

    return Command(goto=goto, update={"next": goto})

In [0]:
messages = [{"role": "system", "content": SUPERVISOR_AGENT_PROMPT},] + [("user", "Get the stock price details of nvidia and intel and display it as a line chart in the same plot comparing the trend")]
response = llm.with_structured_output(Router).invoke(messages)
response["next"]

## Construct Graph with Supervisor Agent and Worker Sub-Agents

We're ready to start building the graph. Below, we define the sub-agents and connect them as nodes to the supervisor agent node

We use the LangGraph built-in `create_react_agent(...)` function to build the two tool-based sub-agents easily

This code defines a financial researcher sub-agent and its node function for use in a LangGraph multi-agent workflow.

- **Agent Creation**:  
  `research_agent` is created using `create_react_agent`, with a set of financial data tools. The `state_modifier` prompt instructs the agent to only search and analyze data, not perform math or coding, and to report back to the supervisor when done.

- **Node Function**:  
  The `research_node` function runs the researcher agent with the current state, prints the last message from the agent for logging, and returns a `Command` to update the workflow state.

- **Message Attribution**:  
  The agent's output is wrapped in a `HumanMessage` with `name="researcher"`. This ensures the message is clearly attributed to the researcher in the conversation history, even though it is generated by an agent. This labeling is important for the supervisor to track which agent produced each message and to maintain a consistent message schema throughout the workflow.

This setup allows the supervisor agent to coordinate tasks between specialized agents, with clear attribution of each message to its source.

In [0]:
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import trim_messages

# Create Financial Researcher Sub-Agent
research_agent = create_react_agent(
    llm, tools=[search_web,
            get_stock_ticker_symbol,
            get_stock_price_metrics,
            get_stock_fundamental_indicator_metrics,
            get_stock_news,
            get_general_market_data], state_modifier="""You are a financial researcher who excels in searching the web and financial platforms and analyzing the data.
                                                        DO NOT do any math or coding.
                                                        Once your task is done report back to the supervisor."""
)

# create node function for financial researcher sub-agent
def research_node(state: State) -> Command[Literal["supervisor"]]:
    result = research_agent.invoke(state)
    print(f"Message from researcher: {result['messages'][-1].content}")
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="researcher")
            ]
        },
        goto="supervisor",
    )

This code defines a coder sub-agent and its node function for use in a LangGraph multi-agent workflow.

- **Agent Creation**:  
  `code_agent` is created using `create_react_agent`, with a Python REPL tool. The `state_modifier` prompt instructs the agent to write and run Python code, visualize charts, and only extract relevant data before coding or plotting. The agent is told to report back to the supervisor when done.

- **Node Function**:  
  The `code_node` function runs the coder agent with the current state, prints the last message from the agent for logging, and returns a `Command` to update the workflow state.

- **Message Attribution**:  
  The agent's output is wrapped in a `HumanMessage` with `name="coder"`. This ensures the message is clearly attributed to the coder in the conversation history, even though it is generated by an agent. This labeling is important for the supervisor to track which agent produced each message and to maintain a consistent message schema throughout the workflow.

This setup allows the supervisor agent to coordinate tasks between specialized agents, with clear attribution of each message to its source.

In [0]:
# Create Coder Sub-Agent
code_agent = create_react_agent(llm, tools=[python_repl_tool], state_modifier="""You are a coder who can write and run python code and also visualize charts and graphs.
                                                                                 Only extract the most relevant data related to the question before running code or creating graphs.
                                                                                 Once your task is done report back to the supervisor.""")

# create node function for coder sub-agent
def code_node(state: State) -> Command[Literal["supervisor"]]:
    result = code_agent.invoke(state)
    print(f"Message from Coder Agent: {result['messages'][-1].content}")
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="coder")
            ]
        },
        goto="supervisor",
    )

In [0]:
# build the agent graph
builder = StateGraph(State)
builder.add_edge(START, "supervisor")
builder.add_node("supervisor", supervisor_node)
builder.add_node("researcher", research_node)
builder.add_node("coder", code_node)
graph = builder.compile()

In [0]:
from IPython.display import display, Image

display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

## Invoke the Team and Test Agent

With the graph created, we can now invoke it and see how it performs!

In [0]:
from IPython.display import Markdown

def call_multi_agent_system(agent, prompt):
    events = agent.stream(
        {"messages": [("user", prompt)]},
        {"recursion_limit": 150},
        stream_mode="values",
    )

    for event in events:
        # event["messages"][-1]
        event["messages"][-1].pretty_print()

    display(Markdown(event["messages"][-1].content))

In [0]:
query = """Get the stock price details of nvidia and intel
           and display it as a line chart in the same plot comparing the trend"""
call_multi_agent_system(graph, query)

In [0]:
query = """find the top 10 companies with the largest market cap and plot it as a bar chart"""
call_multi_agent_system(graph, query)

In [0]:
query = """get the ROE values for nvidia, apple, intel, microsoft, amazon and plot it as a bar chart"""
call_multi_agent_system(graph, query)